Import Libraries

In [6]:
# For text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# For topic modeling
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd

# Download NLTK Resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Load the Data

In [9]:
df = pd.read_csv('npr.csv')
documents = df['Article'].tolist()

Preprocess the Data

In [12]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]
print(preprocessed_documents[0])

['washington', '2016', 'even', 'policy', 'bipartisan', 'politics', 'sense', 'year', 'show', 'little', 'sign', 'ending', 'president', 'obama', 'moved', 'sanction', 'russia', 'alleged', 'interference', 'election', 'concluded', 'republican', 'long', 'called', 'similar', 'severe', 'measure', 'could', 'scarcely', 'bring', 'approve', 'house', 'speaker', 'paul', 'ryan', 'called', 'obama', 'measure', 'appropriate', 'also', 'overdue', 'prime', 'example', 'administration', 'ineffective', 'foreign', 'policy', 'left', 'america', 'weaker', 'eye', 'gop', 'leader', 'sounded', 'much', 'theme', 'urging', 'president', 'obama', 'year', 'take', 'strong', 'action', 'deter', 'russia', 'worldwide', 'aggression', 'including', 'operation', 'wrote', 'devin', 'nunes', 'chairman', 'house', 'intelligence', 'committee', 'week', 'left', 'office', 'president', 'suddenly', 'decided', 'stronger', 'measure', 'indeed', 'appearing', 'cnn', 'frequent', 'obama', 'critic', 'trent', 'frank', 'called', 'much', 'tougher', 'acti

Create document-term matrix

In [15]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents)

# Filter out tokens that appear in less than 15 documents or more than 50% of the documents
dictionary.filter_extremes(no_below=15, no_above=0.5)

# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents] 

Run LDA

In [18]:
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15) # Train an LDA model on the corpus with 2 topics using Gensim's LdaModel class

Interpret Results

In [21]:
# empty list to store dominant topic labels for each document
article_labels = []

# iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
 # for each document, convert to bag-of-words representation
 bow = dictionary.doc2bow(doc)
 # get list of topic probabilities
 topics = lda_model.get_document_topics(bow)
 # determine topic with highest probability
 dominant_topic = max(topics, key=lambda x: x[1])[0]
 # append to the list
 article_labels.append(dominant_topic)
    
# Create DataFrame
df_result = pd.DataFrame({"Article": documents, "Topic": article_labels})

# Print the DataFrame
print("Table with Articles and Topic:")
print(df_result)
print()

Table with Articles and Topic:
                                                 Article  Topic
0      In the Washington of 2016, even when the polic...      0
1        Donald Trump has used Twitter  —   his prefe...      0
2        Donald Trump is unabashedly praising Russian...      0
3      Updated at 2:50 p. m. ET, Russian President Vl...      0
4      From photography, illustration and video, to d...      3
...                                                  ...    ...
11987  The number of law enforcement officers shot an...      4
11988    Trump is busy these days with victory tours,...      2
11989  It’s always interesting for the Goats and Soda...      3
11990  The election of Donald Trump was a surprise to...      2
11991  Voters in the English city of Sunderland did s...      0

[11992 rows x 2 columns]



In [23]:
# Print top terms for each topic
for topic_id in range(lda_model.num_topics):
 print(f"Top terms for Topic #{topic_id}:")
 top_terms = lda_model.show_topic(topic_id, topn=10)
 print([term[0] for term in top_terms])
 print()


Top terms for Topic #0:
['trump', 'president', 'government', 'country', 'state', 'security', 'house', 'obama', 'russia', 'china']

Top terms for Topic #1:
['health', 'percent', 'school', 'state', 'study', 'care', 'student', 'child', 'company', 'program']

Top terms for Topic #2:
['trump', 'clinton', 'republican', 'campaign', 'state', 'vote', 'voter', 'election', 'president', 'candidate']

Top terms for Topic #3:
['know', 'thing', 'think', 'life', 'really', 'world', 'story', 'day', 'work', 'go']

Top terms for Topic #4:
['police', 'court', 'city', 'state', 'law', 'case', 'report', 'right', 'told', 'woman']



In [27]:
# Print the top terms for each topic with weight
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
 print(f"Topic {idx}:")
 terms = [term.strip() for term in topic.split("+")]
 for term in terms:
     weight, word = term.split("*")
     print(f"- {word.strip()} (weight: {weight.strip()})")
 print()

Top Terms for Each Topic:
Topic 0:
- "trump" (weight: 0.014)
- "president" (weight: 0.010)
- "government" (weight: 0.006)
- "country" (weight: 0.005)
- "state" (weight: 0.005)
- "security" (weight: 0.004)
- "house" (weight: 0.004)
- "obama" (weight: 0.004)
- "russia" (weight: 0.004)
- "china" (weight: 0.004)

Topic 1:
- "health" (weight: 0.009)
- "percent" (weight: 0.007)
- "school" (weight: 0.006)
- "state" (weight: 0.005)
- "study" (weight: 0.005)
- "care" (weight: 0.005)
- "student" (weight: 0.004)
- "child" (weight: 0.004)
- "company" (weight: 0.004)
- "program" (weight: 0.004)

Topic 2:
- "trump" (weight: 0.021)
- "clinton" (weight: 0.012)
- "republican" (weight: 0.008)
- "campaign" (weight: 0.007)
- "state" (weight: 0.007)
- "vote" (weight: 0.006)
- "voter" (weight: 0.005)
- "election" (weight: 0.005)
- "president" (weight: 0.005)
- "candidate" (weight: 0.005)

Topic 3:
- "know" (weight: 0.005)
- "thing" (weight: 0.005)
- "think" (weight: 0.005)
- "life" (weight: 0.004)
- "really